In [10]:
import import_ipynb
from frame_slicer import save_frames
import image_process
import video_slicer as vs
import os

In [12]:
parent_folder_dir = 'c:\\ANN_Project\\test\\'
videos_folder_dir = os.path.join(parent_folder_dir, 'video')
videos_name_list = [i for i in os.walk(videos_folder_dir)][0][2]
json_folder_dir = os.path.join(parent_folder_dir, 'json')
json_name_list = [i for i in os.walk(videos_folder_dir)][0][2]

if videos_name_list:
    for video_name, json_name in zip(json_name_list, videos_name_list):
        print('found video: ' + video_name)
        print('found json: ' + json_name)

        inputvideo_dir = os.path.join(videos_folder_dir, video_name)
        os.makedirs(os.path.join(parent_folder_dir, 'resized'), exist_ok=True)
        outputvideo_dir = os.path.join(parent_folder_dir, 'resized', video_name.split('.')[0]) + '_resized.mp4'
        # cut video
        
        vs.convert_and_resize_video(inputvideo_dir, outputvideo_dir, 405, 720)  # resizing

        outputimgs_dir = os.path.join(parent_folder_dir, 'result_img', video_name.split('.')[0])
        os.makedirs(outputimgs_dir, exist_ok=True)
        save_frames(outputvideo_dir, outputimgs_dir)                            # video to img
        img = image_process.ImageProcess()
        img.removeBlurImage(outputimgs_dir, 29, print_img=False)    # remove the blurred images


found video: NIA_SL_FS0001_CROWD18_F.mp4
found json: NIA_SL_FS0001_CROWD18_F.mp4
Frames saved to c:\ANN_Project\test\result_img\NIA_SL_FS0001_CROWD18_F.
95  images deleted
found video: NIA_SL_FS0001_CROWD19_F.mp4
found json: NIA_SL_FS0001_CROWD19_F.mp4
Frames saved to c:\ANN_Project\test\result_img\NIA_SL_FS0001_CROWD19_F.
95  images deleted
